In [31]:
import pandas as pd
import numpy as np
import re
import toolz.itertoolz as itz
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm
import requests as r
from concurrent.futures import ThreadPoolExecutor
import threading


pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)

## Occupancy Extraction

In [32]:
# Functions

# Gets the offset for counting amino acids from column labeled 'Start and End Residues In Protein'
def get_offset(startend): return int(re.search(r'\d+', startend).group())

# Fragments the column 'Full Sequence' value based on mod ("[" and "]") locations and outputs the amino acid and mod
def get_mod_locs(seq, offset=0):
    if 'Cu[I]' in seq:
        seq = seq.replace('Cu[I]', 'Cu(I)')        
        #print(parts)
    mods = []
    for peptide in seq.split('|'):
        peptide_offset = offset
        parts = re.split("\[|\]", str(peptide))
        try: 
            if '' == parts[0]:
                #mods+=[[parts[2][0]+str(offset), parts[1]]]
                mods+=[[str(peptide_offset), parts[1]]]
                parts=parts[2:]
            for l1, l2 in itz.partition(2, parts):
                #mods+=[[l1[-1] + str(offset+len(l1)-1), l2]]
                mods+=[[str(peptide_offset+len(l1)-1), l2]]
                peptide_offset+=len(l1)
        except:
            print('Error Here:', seq, parts)
        
    if mods: return mods
    else: return [[None, None]]


# Extract the range values of the column 'Start and End Residues In Protein' as integers
def get_range_values(range_str):
    return [int(i) for i in re.findall('\d+', range_str)]

def get_prot_seq(accession):
    return r.get('https://rest.uniprot.org/uniprotkb/{}?fields=sequence'.format(accession)).json()['sequence']['value']

# Creates a dictionary structured as 'Protein Accession'(str) -> 'amino acid number'(int) -> {'mods': {'mod'(str): count(int)} , 'total': count(int)}
def prot_get_occupancy(df, nthreads=1): # NOTE: aminoacid numbers are wrong!!
    grouped_df = df.copy()
    grouped_df['Protein Accession'] = (df[df['QValue']<=0.01]['Protein Accession']
                                       .transform(lambda x: re.split("\||;", x))
                                       .explode('Protein Accession'))
    grouped_df = grouped_df.groupby('Protein Accession')
    ngroups = grouped_df.ngroups
    split_groups_indices = [(min(i), max(i)) for i in np.array_split(np.arange(ngroups), nthreads)]
    
    
    def df_subset_handler(g):
        occ = {}
        protein = g['Protein Accession'].values[0]
        
        if protein.startswith('DECOY_'):
            return
        else:
            occ[protein] = {'sequence':get_prot_seq(protein), 'amino_acids':{}}
            ranges = {}

            for index, row in g.iterrows():
                #offset = occ[protein]['sequence'].index(row['Base Sequence'])+1
                offset = 0
                mods = list(get_mod_locs(row['Full Sequence'], offset=offset))
                aa_range = (offset, offset+len(row['Base Sequence']))

                #Keep track of ranges to get total counts of amino acids seen
                if aa_range not in ranges:
                    ranges[aa_range] = {'count':0, 
                                        'values':aa_range}
                ranges[aa_range]['count']+=(row['Full Sequence'].count('|')+1)

                #Add/count mods to protein dict
                for mod in mods:
                    if mod[0]:
                        if mod[0] not in occ[protein]:
                            occ[protein]['amino_acids'][mod[0]] = {'mods':{mod[1]:0}, 'total':0}
                        elif mod[1] not in occ[protein]['amino_acids'][mod[0]]['mods']:
                            occ[protein]['amino_acids'][mod[0]]['mods'][mod[1]]=0
                        occ[protein]['amino_acids'][mod[0]]['mods'][mod[1]]+=1

            #Add total aa observation count
            for aa in occ[protein]['amino_acids']:
                for r in ranges:
                    if ranges[r]['values'][0] <= float(aa) <= ranges[r]['values'][1]:
                        occ[protein]['amino_acids'][aa]['total']+=ranges[r]['count']

        return occ
        
    
    groups = list(grouped_df.groups.keys())
    with ThreadPoolExecutor(max_workers=nthreads) as ex:
        occs = ex.map(df_subset_handler, [grouped_df.get_group(g) for g in groups])
        
    return occs

# Creates a summary output for a given protein similar to Metamorpheus' occupancy report in the AllProteinGroups.tsv file
def protein_occupancy_summary(occupancy_dict, protein_accession):
    for aa in tqdm(sorted(occupancy_dict[protein_accession])):
        for mod in occupancy_dict[protein_accession]['amino_acids'][aa]['mods']:
            occ=occupancy_dict[protein_accession]['amino_acids'][aa]['mods'][mod]/occupancy_dict[protein_accession]['amino_acids'][aa]['total']
            print('aa#{} [{}]: Occupancy={:.2f}({}/{})'.format(aa, mod, occ, occupancy_dict[protein_accession]['amino_acids'][aa]['mods'][mod], occupancy_dict[protein_accession][aa]['total']))

# Uses a string as a randomization seed and outputs random rgba values for that string
def string2rgba(string):
    random.seed(string)
    vals = [random.random() for i in range(4)]
    return vals

In [33]:
# Open file as dataframe

fpath = r"C:\Users\Peter\Desktop\2024-08-07-16-45-12\Task1-SearchTask\AllPSMs.psmtsv"
df_psms = pd.read_csv(fpath, sep='\t', low_memory=False)


fpath2 = r"C:\Users\Peter\Desktop\2024-08-07-16-45-12\Task1-SearchTask\AllQuantifiedProteinGroups.tsv"
df_pgs = pd.read_csv(fpath2, sep='\t', low_memory=False)

In [34]:
df_test = df_psms.sample(n=1000)

In [35]:
nthreads = 5

occ = prot_get_occupancy(df_test, nthreads=nthreads)
occ

<generator object Executor.map.<locals>.result_iterator at 0x000001580EB95E40>

In [36]:
occ = [i for i in occ]
occ

[{'A0A286XX63': {'sequence': 'MAATATMATSGSARKRLLKEEDMTKVEFETSEEVDVTPTFDTMGLREDLLRGIYAYGFEKPSAIQQRAIKQIIKGRDVIAQSQSGTGKTATFSISVLQCLDIQVRETQALILAPTRELAVQIQKGLLALGDYMNVQCHACIGGTNVGEDIRKLDYGQHVVAGTPGRVFDMIRRRSLRTRAIKMLVLDEADEMLNKGFKEQIYDVYRYLPPATQVVLISATLPHEILEMTNKFMTDPIRILVKRDELTLEGIKQFFVAVEREEWKFDTLCDLYDTLTITQAVIFCNTKRKVDWLTEKMREANFTVSSMHGDMPQKERESIMKEFRSGASRVLISTDVWARGLDVPQVSLIINYDLPNNRELYIHRIGRSGRYGRKGVAINFVKNDDIRILRDIEQYYSTQIDEMPMNVADLI',
   'amino_acids': {}}},
 {'A0A2I3RIL2': {'sequence': 'MAKMEVKTSLLDNMIGVGDMVLLEPLNEETFINNLKKRFDHSEIYTYIGSVVISVNPYRSLPIYSPEKVEEYRNRNFYELSPHIFALSDEAYRSLRDQDKDQCILITGESGAGKTEASKLVMSYVAAVCGKGAEVNQVKEQLLQSNPVLEAFGNAKTVRNDNSSRFGKYMDIEFDFKGDPLGGVISNYLLEKSRVVKQPRGERNFHVFYQLLSGASEELLNKLKLERDFSRYNYLSLDSAKVNGVDDAANFRTVRNAMQIVGFMDHEAESVLAVVAAVLKLGNIEFKPESRVNGLDESKIKDKNELKEICELTGIDQSVLERAFSFRTVEAKQEKVSTTLNVAQAYYARDALAKNLYSRLFSWLVNRINESIKAQTKVRKKVMGVLDIYGFEIFEDNSFEQFIINYCNEKLQQIFIELTLKEEQEEYIREDIEWTHIDYFNNAIICDLIENNTNGILAMLDEECLRPGTVTDETFLEKLNQVCATHQHFESRMSKCSRFLND

In [ ]:
#protein_occupancy_summary(occ, 'Q9ERD7') # Specify 'Protein Accession' here

In [ ]:
#df2 = df[(df['Protein Accession']=='P55095') & (df['QValue']<=0.01)]

## Occupancy Visualization

In [ ]:
protein = 'P52480' # Specify 'Protein Accession' here. Another good one is P52480.
n = len(occ[protein])
shape = (n//2+n%2, 2)

fig, axes = plt.subplots(*(shape), figsize=(20,n*2), layout='tight')

axes = np.ravel(axes)
fig.subplots_adjust(wspace=0, hspace=0)
if n//2!=0:
    axes[-1].set(visible=False)

for ind, aa in enumerate(sorted(occ[protein])):
    axes[ind].set_title('aa#{}'.format(aa))
    values = list(np.array(list(occ[protein][aa]['mods'].values()))/occ[protein][aa]['total'])
    labels = occ[protein][aa]['mods'].keys()
    if np.sum(values)!=1:
        values=[1-np.sum(values)]+values
        labels = ['Unmodified', *occ[protein][aa]['mods'].keys()]
    
    pie = axes[ind].pie(values,
                        #labels=labels, 
                        normalize=False, 
                        wedgeprops={'alpha':0.7}, 
                        colors=[string2rgba(i) for i in labels], 
                        autopct=lambda x: '{:.0f}%'.format(x),
                        pctdistance=1.2)
    axes[ind].legend(pie[0], 
                     labels, 
                     title='Modifications',
                     loc='center left', 
                     bbox_to_anchor=(1, 0, 0.5, 1))


In [125]:
gb = df_psms.groupby('Protein Accession')
x = sorted(list(gb.groups.keys()), key=lambda x: len(x))[:10]
pd.concat([gb.get_group(i) for i in x])

,File Name,Scan Number,Scan Retention Time,Num Experimental Peaks,Total Ion Current,Precursor Scan Number,Precursor Charge,Precursor MZ,Precursor Mass,Score,Delta Score,Notch,Base Sequence,Full Sequence,Essential Sequence,Ambiguity Level,"PSM Count (unambiguous, <0.01 q-value)",Mods,Mods Chemical Formulas,Mods Combined Chemical Formula,Num Variable Mods,Missed Cleavages,Peptide Monoisotopic Mass,Mass Diff (Da),Mass Diff (ppm),Protein Accession,Protein Name,Gene Name,Organism Name,Identified Sequence Variations,Splice Sites,Contaminant,Decoy,Peptide Description,Start and End Residues In Protein,Previous Amino Acid,Next Amino Acid,Theoreticals Searched,Decoy/Contaminant/Target,Matched Ion Series,Matched Ion Mass-To-Charge Ratios,Matched Ion Mass Diff (Da),Matched Ion Mass Diff (Ppm),Matched Ion Intensities,Matched Ion Counts,Normalized Spectral Angle,Localized Scores,Improvement Possible,Cumulative Target,Cumulative Decoy,QValue,Cumulative Target Notch,Cumulative Decoy Notch,QValue Notch,PEP,PEP_QValue
11421,130328_ArgC_Frac8,14691,154.43187,200.0,6.573518e+04,14686,2.0,892.48657,1782.95859,8.053,0.905,0,DIFQLSLNTINKYSK,DIFQLSLNTINKYSK,DIFQLSLNTINKYSK,1,0,NaN,NaN,NaN,0,1,1782.95160,0.00699,3.92,A0EDQ7,"Chromosome undetermined scaffold_90, whole gen...",primary:GSPATT00025768001,Paramecium tetraurelia,NaN,NaN,N,N,full,[193 to 207],R,I,,T,"[y1+1, y2+1, y3+1, y8+1, y10+1];[b2+1, b3+1, b...","[y1+1:147.11198, y2+1:234.14310, y3+1:397.2046...","[y1+1:-0.00082, y2+1:-0.00174, y3+1:-0.00348, ...","[y1+1:-5.61, y2+1:-7.44, y3+1:-8.78, y8+1:-7.9...","[y1+1:1568, y2+1:283, y3+1:119, y8+1:362, y10+...",8,-1.0,,,9872,1473.716026,0.149282,9259,1022.566026,0.110440,0.999970,0.300890
15904,130328_ArgC_Frac8,6195,81.46477,171.0,2.924947e+06,6194,3.0,608.03760,1821.09096,7.080,0.008,1,KVTGSVLIGPTPVNIIGR,KVTGSVLIGPTPVNIIGR,KVTGSVLIGPTPVNIIGR,1,0,NaN,NaN,NaN,0,1,1820.08837,1.00260,550.85,A0PEK7,Pol protein (Fragment),primary:pol,Human immunodeficiency virus 1,NaN,NaN,N,N,full,[70 to 87],K,D,,T,"[b1+1, b2+1, b7+2, b8+2, b9+1];[y1+1, y9+1]","[b1+1:129.10195, b2+1:228.17007, b7+2:343.2203...","[b1+1:-0.00029, b2+1:-0.00058, b7+2:0.00920, b...","[b1+1:-2.24, b2+1:-2.55, b7+2:13.45, b8+2:10.9...","[b1+1:99229, b2+1:5203, b7+2:13655, b8+2:8251,...",7,-1.0,,,12205,3570.869618,0.292431,1424,1317.088095,0.913438,1.000000,0.759114
9357,130328_ArgC_Frac8,11357,123.84906,128.0,1.529614e+06,11353,2.0,676.89995,1351.78535,8.237,0.053,0,LAVIKEFLYTR,LAVIKEFLYTR,LAVIKEFLYTR,1,0,NaN,NaN,NaN,0,1,1351.78637,-0.00102,-0.76,A0Q4P2,"Transporter-associated protein, HlyC/CorC family",primary:FTN_0301,Francisella tularensis subsp. novicida (strain...,NaN,NaN,N,N,full,[243 to 253],K,Y,,T,"[y1+1, y3+1, y5+1, y8+1, y9+1];[b2+1, b3+1, b6+1]","[y1+1:175.11819, y3+1:439.22814, y5+1:699.3808...","[y1+1:-0.00076, y3+1:-0.00182, y5+1:-0.00158, ...","[y1+1:-4.35, y3+1:-4.16, y5+1:-2.26, y8+1:-4.6...","[y1+1:45544, y3+1:20084, y5+1:71248, y8+1:3641...",8,-1.0,,,8699,605.932692,0.069591,8413,458.182692,0.054410,0.999998,0.545752
9373,130328_ArgC_Frac8,11348,123.76234,136.0,1.828329e+06,11342,2.0,676.89946,1351.78436,8.230,0.052,0,LAVIKEFLYTR,LAVIKEFLYTR,LAVIKEFLYTR,1,0,NaN,NaN,NaN,0,1,1351.78637,-0.00201,-1.49,A0Q4P2,"Transporter-associated protein, HlyC/CorC family",primary:FTN_0301,Francisella tularensis subsp. novicida (strain...,NaN,NaN,N,N,full,[243 to 253],K,Y,,T,"[y1+1, y3+1, y5+1, y8+1, y9+1];[b2+1, b3+1, b6+1]","[y1+1:175.11819, y3+1:439.22823, y5+1:699.3808...","[y1+1:-0.00076, y3+1:-0.00173, y5+1:-0.00164, ...","[y1+1:-4.35, y3+1:-3.95, y5+1:-2.35, y8+1:-4.6...","[y1+1:53430, y3+1:22405, y5+1:81799, y8+1:4340...",8,-1.0,,,8712,608.932692,0.069896,8426,461.182692,0.054733,0.999999,0.606443
32083,130328_ArgC_Frac8,639,13.20432,164.0,2.880061e+04,637,3.0,325.19252,972.55572,5.195,0.011,1,KPQNTTKR,KPQNTTKR,KPQNTTKR,1,0,NaN,NaN,NaN,0,2,971.55123,1.00449,1033.91,A0Q4Y8,Uncharacterized protein,primary:FTN_0400,Francisella tularensis subsp. novicida (strain...,NaN,NaN,N,N,full,[16 